## 作業
礙於不是所有同學都有 GPU ，這邊的範例使用的是簡化版本的 ResNet，確保所有同學都能夠順利訓練!


最後一天的作業請閱讀這篇非常詳盡的[文章](https://blog.gtwang.org/programming/keras-resnet-50-pre-trained-model-build-dogs-cats-image-classification-system/)，基本上已經涵蓋了所有訓練　CNN 常用的技巧，請使用所有學過的訓練技巧，盡可能地提高 Cifar-10 的 test data 準確率，截圖你最佳的結果並上傳來完成最後一次的作業吧!

另外這些技巧在 Kaggle 上也會被許多人使用，更有人會開發一些新的技巧，例如使把預訓練在 ImageNet 上的模型當成 feature extractor 後，再拿擷取出的特徵重新訓練新的模型，這些技巧再進階的課程我們會在提到，有興趣的同學也可以[參考](https://www.kaggle.com/insaff/img-feature-extraction-with-pretrained-resnet)

In [1]:
from keras.datasets import cifar10
from keras.utils import to_categorical

#讀取資料集並進行資料前處理
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

#輸入圖像維度
#input_shape = x_train.shape[1:]

#將影像縮放到 0-1 之間
x_train = x_train / 255.
x_test = x_test / 255.

#對label做one-hot encoding
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [2]:
print(x_train.shape, y_train.shape)

(50000, 32, 32, 3) (50000, 10)


In [3]:
from keras.applications.resnet50 import ResNet50

IMAGE_SIZE = (224, 224)

model_resnet50 = ResNet50(include_top=False, 
                          weights='imagenet',
                          input_tensor=None,
                          input_shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3))

In [4]:
input_shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3)
print(input_shape)

(224, 224, 3)


In [5]:
from tensorflow.python.keras.layers import Flatten, Dense, Dropout

NUM_CLASSES = 2

x = model_resnet50.output
x = Flatten()(x)
x = Dropout(0.5)(x)
output_layer = Dense(NUM_CLASSES, activation='softmax', name='softmax')(x)

In [6]:
from tensorflow.python.keras.models import Model

#凍結網路層數
FREEZE_LAYERS = 2

#設定凍結與要進行訓練的網路層
model_resnet50_final = Model(inputs=model_resnet50.input, outputs=output_layer)
for layer in model_resnet50_final.layers[:FREEZE_LAYERS]:
    layer.trainable = False
for layer in model_resnet50_final.layers[FREEZE_LAYERS:]:
    layer.trainable = True

In [7]:
from keras.optimizers import Adam

model_resnet50_final.compile(optimizer=Adam(lr=1e-5),
                             loss='categorical_crossentropy', 
                             metrics=['accuracy'])

In [8]:
model_resnet50_final.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
_______________________________________________________________________________________

In [9]:
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   channel_shift_range=10,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

In [ ]:
batch_size = 8
epochs = 10

history = model_resnet50_final.fit_generator(train_datagen.flow(x_train, y_train, batch_size=batch_size),
                    steps_per_epoch=int(len(x_train)/batch_size), # 一個 epochs 要執行幾次 update，通常是資料量除以 batch size
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))

score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/10


In [ ]:
batch_size = 8 # batch 的大小，如果出現 OOM error，請降低這個值
num_classes = 10 # 類別的數量，Cifar 10 共有 10 個類別
epochs = 10 # 訓練整個資料集共 30個循環
#30跑太久, 改成10
model_resnet50_final.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

history = model_resnet50_final.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))
score = model_resnet50_final.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])